In [ ]:
import numpy as np
## INITIALIZATION ##

# Environment #
W = np.array([[17.96208853,17.56368787,20.61838065,25.18334004,22.52074534,26.77293274,25.57287022,20.51386757,18.42710807,21.36785401,18.12489511,14.94887125, 12.87491946, 13.09745288,  7.21907596, -1.459937 ,   7.740678 ,  10.37488769,  4.48765114  ,5.46872134, -1.68147061 , 4.22306641, 17.8954532  , 2.58745642, 1.11793932, -3.81641426, -3.75646  ,  -2.45822383 ,-0.22924514, -2.24006629],[-11.14862732 ,-13.8301893 , -17.73220257,  -8.10191481 , -2.33771849,  -0.71133681  , 7.3276312  ,  2.85030499 , -2.26370296 , -2.01329856,  -3.88281926 , -0.60767267,  -3.66318881 , -3.93941925,  -1.41578766,   8.08154894 , 10.92129449  , 6.36317964,   6.27415534,  -6.86086859,  -4.49156021  ,-6.23473274 , -7.2650244 ,   0.06751707  ,-7.68552073,  -4.35247244  , 0.49549029 , -4.15317153 , -0.96148253 , -2.29360969]])
ch = 1.225
cz = 2.256E-5
ce = 4.2559
cf = ce/2+1
rho = lambda z :ch*(1-z*cz)**ce
z0 = 1200
rho0 = ch*(1-z0*cz)**ce

# Initial conditions #
N = 31
zf = 0
rz0 = -7.9
t0 = 0
vz0 = 18.5
x_0 = np.array([[400], [400]])
psi_0 = 0.
z = lambda t: 1/cz*(1-((((1-z0*cz)**cf)/cf/cz -(t-t0)*rz0 *  np.sqrt(rho0)/ np.sqrt(ch))*cf*cz)**(1/cf) )
tf = t0 + np.sqrt(ch)/rz0/ np.sqrt(rho0) * (((1-z0*cz)**cf)/cf/cz - ((1-zf*cz)**cf)/cf/cz)
dt = tf / (N - 1)  # constant time step
time =  np.linspace(0, tf, N)


# Dynamics #
A = np.eye(2, 2)
B_p = np.eye(2, 2) * dt * 0.5
B_m = np.eye(2, 2) * dt * 0.5
phid_max = 0.14  # maximum rate of turn
zt = z(time)
vt = lambda z:vz0*np.sqrt(rho0/rho(z))
v = vt(zt)
u_0 = np.array([[v[0]*np.cos(psi_0)], [v[0]*np.sin(psi_0)]])

# Numerical Parameters #
eps_h_val = 0.1
eps_convergence = 0.01
alpha_1 = 100
alpha_2 = 10
alpha_3 = 1

In [ ]:
import cvxpy as cvx
## OPTIMIZATION ##

# Variables #
x = cvx.Variable((2, N))
u = cvx.Variable((2, N))
eps_h = cvx.Variable(nonneg = True)
u_bar = cvx.Parameter((2, N))
u_init = np.array([v*np.cos(psi_0),v*np.sin(psi_0)])
u_bar.value = np.divide(u_init,np.linalg.norm(u_init, axis=0))

# Constraints #
const = [x[:, [0]] == x_0]
const += [u[:, [0]] == u_0]
const += [x[:, [k + 1]] == A @ x[:, [k]] + (B_m @ u[:, [k]] + B_p @ u[:, [k + 1]])+[W[:,k]] for k in range(0, N - 1)]  # constraint on the dynamics
const += [(cvx.norm2(cvx.diff(u, axis=1), axis=0) / dt / v[k])[k] <= phid_max for k in range(0, N - 1)]

# LINEARIZATION CONSTRAINTS
const += [ u_bar[:,[k]].T @ u[:,[k]] - v[k] >= -eps_h for k in range(0, N)]
const += [ cvx.norm(u[:,[k]]) - v[k] <= eps_h for k in range(0, N)]

final_position = cvx.norm(x[:, [-1]])
final_angle = 2- u[1, [-1]]/np.linalg.norm(v[-1])
control_cost = cvx.sum_squares(cvx.norm(cvx.diff(u, axis=1),axis=0)/v[0:N-1])/dt
cost = alpha_1*final_position + alpha_2*final_angle + control_cost
MAX_ITER = 50

it_final_position = np.empty((MAX_ITER))
it_final_angle=np.empty((MAX_ITER))
it_control_cost =np.empty((MAX_ITER))
it_cost =np.empty((MAX_ITER))
X = np.empty((2,N,MAX_ITER))
U = np.empty((2,N,MAX_ITER))
D = np.empty((N-1,MAX_ITER))

problem = cvx.Problem(cvx.Minimize(cost), const + [eps_h ==0.1])
first_stage_converged = False

print('Iteration number\t Final position\t Final angle\t Control cost\t Total cost') 
for i in range(MAX_ITER):

    s = problem.solve(solver=cvx.ECOS, verbose=True, warm_start=True)
    u_bar.value = np.divide(u.value,np.linalg.norm(u.value, axis=0))

    x_star = x.value
    u_star = u.value
    
    X[:,:,i] = x_star
    U[:,:,i] = u_star
    
    it_final_position[i] = final_position.value
    it_final_angle[i] = final_angle.value
    it_control_cost[i] = control_cost.value
    it_cost[i] = cost.value

    print(str(i)+'\t'+'\t'+'\t'+"%10.3E" %it_final_position[i]+'\t'+"%10.3E" %it_final_angle[i]+'\t'+"%10.3E" %it_control_cost[i]+'\t'+"%10.3E" %it_cost[i])
    if (np.abs(it_cost[i]-it_cost[i-1]) < eps_convergence) and first_stage_converged:
        print("STAGE 2 CONVERGED AFTER " +str(i)+" ITERATIONS")
        n_iter = i
        break
    if (i>1) and (np.abs(it_cost[i]-it_cost[i-1]) < eps_convergence) and not first_stage_converged:
        print("STAGE 1 CONVERGED AFTER " +str(i)+" ITERATIONS")
        cost = cost+ alpha_3*eps_h
        problem = cvx.Problem(cvx.Minimize(cost), const)
        first_stage_converged = True
        n_iter_first = i
        

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
plt.rcParams.update({"text.usetex": True,'font.size': 10})
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
def cm2inch(*tupl):
    inch = 2.54
    if isinstance(tupl[0], tuple):
        return tuple(i / inch for i in tupl[0])
    else:
        return tuple(i / inch for i in tupl)

colormap = cm.get_cmap('inferno', n_iter)
colors_ = [colormap(range(n_iter + 1)[n_iter - j]) for j in range(n_iter + 1)]

fig = plt.figure(figsize=(cm2inch(18, 12)), dpi=80, facecolor='w', edgecolor='k')
gs = fig.add_gridspec(ncols=2, nrows=2, width_ratios = [5,5],height_ratios = [6,6])

ax1 = fig.add_subplot(gs[0,0])
fs = 10
plt.xlabel('Position $x_1 $ [m]', fontsize=fs)
plt.ylabel('Position $x_2 $ [m]', fontsize=fs)
plt.grid(True)
plt.axis('equal')
plt.yticks([-2000, -1000, 0,  1000,2000], rotation=90)
plt.xticks([-2000, -1000, 0,  1000,2000 ])

ax2 = fig.add_subplot(gs[0,1])
plt.hlines(phid_max, time[0], time[-1], color = 'r',linestyle = '--',linewidth=3,zorder=3)
plt.hlines(-phid_max, time[0], time[-1], color = 'r',linestyle = '--',linewidth=3,zorder=3)
plt.xlabel('Time $t$ [s]', fontsize=fs)
plt.ylabel('Reference $\delta(t)$ [rad/s]', fontsize=fs)
plt.grid(True)
plt.yticks([-phid_max,0,phid_max],['$-\dot \psi_{m}$',0,'$\dot \psi_{m}$'],rotation=90)
divider = make_axes_locatable(ax2)
cax = divider.append_axes('right', size='5%', pad=0.05)
cmap = mpl.cm.inferno_r
norm = mpl.colors.Normalize(vmin=1, vmax=n_iter)
cb1 = mpl.colorbar.ColorbarBase(cax, cmap=cmap,norm=norm)
cb1.set_label('Iteration \#',fontsize=fs)


ax3 = fig.add_subplot(gs[1,0])
plt.hlines(eps_h_val, time[0], time[-1], color = 'r',linestyle = '--',linewidth=3,label='$\pm\epsilon_h$',zorder=3)
plt.hlines(-eps_h_val, time[0], time[-1], color = 'r',linestyle = '--',linewidth=3,zorder=3)
plt.ylabel('$||\mathbf{u}||_2-v$ [m/s]', fontsize=fs)
plt.grid(True)
plt.yticks([-eps_h_val,0,eps_h_val],['$-\epsilon_h$',0,'$\epsilon_h$'],rotation=90)
#plt.plot(time, np.linalg.norm(U[:,:,-1],axis=0)-v, color=colors_[-1],linewidth=3,label='_nolegend_')

ax4 = fig.add_subplot(gs[1,1])
plt.semilogy(np.arange(0,n_iter),it_cost[0:n_iter], '-',label="$\sum$", color='k', linestyle='solid',linewidth=1)
plt.semilogy(np.arange(0,n_iter),alpha_1*it_final_position[0:n_iter] , '-',label="$||x_f||$", color='k', linestyle='dashed',linewidth=1)
plt.semilogy(np.arange(0,n_iter),alpha_2*it_final_angle[0:n_iter] , '-',label="$||u_{1f}||$", color='k', linestyle='dotted',linewidth=1)
plt.semilogy(np.arange(0,n_iter),it_control_cost[0:n_iter] , '-',label="$\int \delta_a^2 dt$", color='k', linestyle='dashdot',linewidth=1)
plt.grid(True, which="both", ls="-")
plt.xlabel('Iteration \# [-]', fontsize=fs)
plt.ylabel('Value [-]', fontsize=fs)
plt.gca()
plt.ylim(10E-13, 10E5)
#ax4.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=6)
ax4.legend(loc='lower left',fontsize=fs)
plt.axvline(x=n_iter_first, color="red", linestyle="-")
plt.text(n_iter_first-2, 1E-7, '$1^{st}$ stage',rotation=90,color = 'red') 
plt.text(n_iter_first+0.5, 1E-7, '$2^{nd}$ stage',rotation=90,color = 'red') 


for i in range(n_iter):
    ax1.plot(X[1,:,i],X[0,:,i], '-', color=colors_[i],linewidth=2,label='_nolegend_')   
    D[:,i] = np.cross(np.diff(U[:,:,i], axis=1) / dt, U[:,:N-1,i], axis=0)/v[0:N-1]/v[0:N-1]
    ax2.step(time[:N-1],D[:,i], where = 'post',color=colors_[i],label='_nolegend_')
    ax3.plot(time, np.linalg.norm(U[:,:,i],axis=0)-v, color=colors_[i],linewidth=1,label='_nolegend_')

ax3.plot(time, np.linalg.norm(U[:,:,i],axis=0)-v, color=colors_[i],linewidth=3,label='_nolegend_')
    
plt.tight_layout()
plt.savefig('fig1_paper.pdf', bbox_inches='tight')